# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [29]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2137805163598,
 'week52high': 134.55,
 'week52low': 56.89,
 'week52change': 0.8243402512578206,
 'sharesOutstanding': 17676605608,
 'float': 0,
 'avg10Volume': 98135625,
 'avg30Volume': 94086920,
 'day200MovingAvg': 110.22,
 'day50MovingAvg': 120.02,
 'employees': 0,
 'ttmEPS': 3.42,
 'ttmDividendRate': 0.8086275314208954,
 'dividendYield': 0.006731918396227835,
 'nextDividendDate': '0',
 'exDividendDate': '2020-10-23',
 'nextEarningsDate': '0',
 'peRatio': 36.54202415946612,
 'beta': 1.1851201308666204,
 'maxChangePercent': 46.33235527308984,
 'year5ChangePercent': 3.61648533477115,
 'year2ChangePercent': 2.0088085543106797,
 'year1ChangePercent': 0.8724880725471417,
 'ytdChangePercent': 0.6809985883218457,
 'month6ChangePercent': 0.4265530197279979,
 'month3ChangePercent': 0.04006288661054758,
 'month1ChangePercent': 0.04892564036700101,
 'day30ChangePercent': 0.04894728714451933,
 'day5ChangePercent': -0.010829706221276506}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.8724880725471417

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataFrame = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataFrame = final_dataFrame.append(
        pd.Series(
        [
            symbol,
            data[symbol]['quote']['latestPrice'],
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'
        ],
        index = my_columns),
            ignore_index = True
        )
final_dataFrame

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,119.130,0.450653,N/A
1,AAL,18.190,-0.376219,N/A
2,AAP,163.440,0.0497034,N/A
3,AAPL,126.120,0.841996,N/A
4,ABBV,112.700,0.327689,N/A
...,...,...,...,...
500,YUM,109.082,0.098459,N/A
501,ZBH,150.630,0.0118381,N/A
502,ZBRA,382.640,0.503562,N/A
503,ZION,43.490,-0.107856,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_dataFrame.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataFrame = final_dataFrame[:50]
final_dataFrame.reset_index(inplace = True)

## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio:')
    
    try:
        float(portfolio_size)
    except ValueError:
        print('That is not a number! \nPlease try again:')
        portfolio_size = input('Enter the size of your portfolio:')
portfolio_input()
print(portfolio_size)

Enter the size of your portfolio:1000000
1000000


In [9]:
position_size = float(portfolio_size)/len(final_dataFrame.index)
for i in range(0, len(final_dataFrame)):
    final_dataFrame.loc[i, 'Number of Shares to Buy'] = position_size/final_dataFrame.loc[i, 'Price']
final_dataFrame

/Users/mikereidy/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,38.190,2.24048,523.697
1,345,NVDA,538.740,1.49375,37.1237
2,275,LB,42.050,1.45606,475.624
3,31,AMD,93.210,1.34816,214.569
4,23,ALB,145.090,1.2762,137.845
5,179,FCX,25.311,1.06978,790.17
6,387,PYPL,225.860,1.04654,88.5504
7,339,NOW,559.770,0.978443,35.729
8,180,FDX,297.090,0.952341,67.3197
9,388,QCOM,160.400,0.89192,124.688


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [25]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile'
]
hqm_dataFrame = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataFrame = hqm_dataFrame.append(
            pd.Series(
            [       symbol,
                    data[symbol]['quote']['latestPrice'],
                    'N/A',
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month6ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month3ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month1ChangePercent'],
                    'N/A',
            ],
            index = hqm_columns),
            ignore_index = True
        )
hqm_dataFrame

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,A,119.100,N/A,0.445757,N/A,0.310765,N/A,0.197388,N/A,0.0356332,N/A
1,AAL,18.405,N/A,-0.385502,N/A,-0.0783383,N/A,0.321557,N/A,0.312534,N/A
2,AAP,162.630,N/A,0.0513783,N/A,0.0994217,N/A,0.0254953,N/A,0.00882607,N/A
3,AAPL,127.000,N/A,0.841049,N/A,0.433233,N/A,0.0405348,N/A,0.047506,N/A
4,ABBV,108.500,N/A,0.33322,N/A,0.162005,N/A,0.207501,N/A,0.144843,N/A
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,108.880,N/A,0.0988641,N/A,0.128566,N/A,0.152138,N/A,0.0509805,N/A
501,ZBH,151.120,N/A,0.0114464,N/A,0.10446,N/A,0.053085,N/A,-0.0582001,N/A
502,ZBRA,377.690,N/A,0.509145,N/A,0.413092,N/A,0.483186,N/A,0.0719071,N/A
503,ZION,42.710,N/A,-0.107961,N/A,0.108335,N/A,0.343416,N/A,0.0915706,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [31]:
time_periods = [
        'One-Year',
        'Six-Month',
        'Three-Month',
        'One-Month'
        ]

for row in hqm_dataFrame.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataFrame.loc[row, percentile_col] = score(hqm_dataFrame[change_col], hqm_dataFrame.loc[row, change_col])
hqm_dataFrame

TypeError: '<' not supported between instances of 'NoneType' and 'float'

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [11]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

NameError: name 'writer' is not defined

## Saving Our Excel Output

As before, saving our Excel output is very easy: